In [7]:
%pip install datasets pandas azure-identity "azure-search-documents==11.6.0b1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 287.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 426.0 kB/s eta 0:00:000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.3/315.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 194.7 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import datasets

lang='ja'  # or any of the 16 languages
miracl_corpus = datasets.load_dataset('miracl/miracl-corpus', lang, cache_dir='.cache')['train']
for doc in miracl_corpus:
   docid = doc['docid']
   title = doc['title']
   text = doc['text']

Generating train split: 6953614 examples [03:35, 32335.45 examples/s]


In [ ]:
lang='ja'  # or any of the 16 languages
miracl = datasets.load_dataset('miracl/miracl', lang, use_auth_token=True, cache_dir='.cache')

# training set:
for data in miracl['train']:  # or 'dev', 'testA'
  query_id = data['query_id']
  query = data['query']
  positive_passages = data['positive_passages']
  negative_passages = data['negative_passages']
  
  for entry in positive_passages: # OR 'negative_passages'
    docid = entry['docid']
    title = entry['title']
    text = entry['text']

In [21]:
from azure.identity.aio import DefaultAzureCredential
from azure.search.documents.aio import SearchClient
from azure.search.documents.indexes.aio import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchableField,
    SearchIndex,
    SearchIndexer,
    SearchFieldDataType,
)
import os
ANALYSER = "ja.microsoft"

service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index_name = 'test-miracl-index-ja-microsoft'
azure_cred = DefaultAzureCredential()

search_client = SearchClient(service_endpoint, index_name, azure_cred)
index_client = SearchIndexClient(service_endpoint, azure_cred)
indexer_client = SearchIndexerClient(service_endpoint, azure_cred)

async def create_miracl_corpus_index(name):
    if await index_client.get_index(name):
        return
    fields = [
        SimpleField(name="docid", type=SearchFieldDataType.String, key=True),
        SimpleField(name="title", type=SearchFieldDataType.String),
        SearchableField(name="text", type=SearchFieldDataType.String, analyzer_name=ANALYSER),
    ]
    index = SearchIndex(
        name=name,
        fields=fields)
    result = await index_client.create_index(index)
    return result

async def create_indexer(name):
    # Create an indexer
    indexer_name = f"{name}-indexer"

    indexer = SearchIndexer(
        name=indexer_name,
        description="Indexer to index documents and generate embeddings",
        skillset_name=f"{name}-skillset",
        target_index_name=name,
        data_source_name=f"miracl-cosmos",
    )

    return await indexer_client.create_or_update_indexer(indexer)

await create_miracl_corpus_index(index_name)
await create_indexer(index_name)


HttpResponseError: () This indexer refers to a data source 'test-miracl-index-ja-microsoft-blob' that doesn't exist
Code: 
Message: This indexer refers to a data source 'test-miracl-index-ja-microsoft-blob' that doesn't exist